<a id="top"></a>
# Team RM5 Classification Predict Notebook

**Team Members:** Abednego Pakaree, Iman Mokwena, Lizette Loubser, Omphile Louw, Refiloe Elvis Phipa, Silindokuhle Kubheka    
**Supervisor:** Ridha Moosa

* [Notebook repo](https://github.com/Lizette95/Team_RM5_ClassificationPredict_Notebook)
* [Streamlit repo](https://github.com/Lizette95/classification-predict-streamlit-template)
* [Trello board](https://trello.com/b/pFKnwtGF/classificationteamrm5) 

### Table of Contents
---
1. [Introduction](#intro)
 * Background
 * Problem Statement
---
2. [Load Dependencies](#imports)
---
3. [Exploratory Data Analysis](#EDA)
---
4. [Data Preprocessing](#preprocessing) 
---
5. [Modelling](#modelling)
---
6. [Performance Evaluation](#evaluation)
---
7. [Model Analysis](#analysis)
---
8. [Conclusion](#conclusion)
---

<a id="intro"></a>
## 1. Introduction  

### Project Description


### Problem Statement


<a id="imports"></a>
## 2. Load Dependencies

In [0]:
# Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

# Prerequisites
#import sys
#!{sys.executable} -m pip install scikit-learn

# Load Dependencies
import numpy as np
import pandas as pd
import seaborn as sns

# Display
%matplotlib inline
sns.set(font_scale=1)
sns.set_style("white")

<a id="EDA"></a>
## 3. Exploratory Data Analysis

<a id="preprocessing"></a>
## 4. Data Preprocessing

<a id="modelling"></a>
## 5. Modelling

<a id="evaluation"></a>
## 6. Performance Evaluation

<a id="analysis"></a>
## 7. Model Analysis

<a id="conclusion"></a>
## 8. Conclusion

[Back to top ↑](#top)